In [7]:
import os
import csv
import paramiko
import xml.etree.ElementTree as ET

# Remote server details #
# SNGMDBB14
hostname = 'IP address'
username = 'username'
password = 'pass'

# Directory path containing XML files on the remote server
remote_directory = '/opt/XXX/datafiles/XXX/'
# Local directory to store copy of downloaded XML files
local_directory = './downloaded_xml/'
output_file = 'XXX_OUTPUT.csv'

# Create a Paramiko SSH client
ssh_client = paramiko.SSHClient()

# Automatically add the remote server's host key
ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

# Connect to the remote server
ssh_client.connect(hostname=hostname, username=username, password=password)

# Create the local directory if it doesn't exist
os.makedirs(local_directory, exist_ok=True)

# List to store extracted data
data = []

# Retrieve the XML files from the remote directory
ftp_client = ssh_client.open_sftp()
xml_files = ftp_client.listdir(remote_directory)

# Download and process each XML file
for filename in xml_files:
    if filename.endswith('.xml'):
        remote_file_path = os.path.join(remote_directory, filename)
        local_file_path = os.path.join(local_directory, filename)

        # Download the XML file from the remote server
        ftp_client.get(remote_file_path, local_file_path)

        # Parse the downloaded XML file
        tree = ET.parse(local_file_path)
        root = tree.getroot()

        # Extract desired data from the XML file
        for target_service in root.iter('targetService'):
            for tickers in target_service.iter('tickers'):
                for mapping in tickers.iter('mapping'):
                    sr = mapping.get('sr')
                    records = mapping.get('record')
                    source_dataset = tickers.get('sourceDataset')
                    field_maps = tickers.get('fieldMaps')

                    # Append extracted data to the list
                    data.append([target_service.get('name'), records, sr, source_dataset, field_maps])
# Close the FTP client and SSH connection
ftp_client.close()
ssh_client.close()

# Write the extracted data to the output CSV file
with open(output_file, 'a', newline='') as csvfile:
    writer = csv.writer(csvfile)

    # Write the header row
    writer.writerow(['targetService','record', 'sr', 'sourceDataset', 'fieldMaps'])

    # Write the data rows
    writer.writerows(data)

print('Data has been appended to', output_file)


Data has been appended to output.csv
